## Data cleaning & EDA

In [27]:
import re
import pandas as pd

In [ ]:
df_benign = pd.read_csv('../Data/CSV_benign.csv')
df_malware = pd.read_csv('../Data/CSV_malware.csv')

# 'Country' column name is duplicated in malware csv, therefore I decided to rename both. While reading it, pandas reads duplicated column name with '.1' suffix
df_benign.rename(columns={'Country.1':'Country_1'}, inplace=True)
df_malware.rename(columns={'Country.1':'Country_1'}, inplace=True)

# Reindex columns
df_benign = df_benign.reindex(sorted(df_benign.columns), axis=1)
df_malware = df_malware.reindex(sorted(df_malware.columns), axis=1)
# df_malware.head(3)

In [ ]:
# By digging into missmatch in dtypes, I was able to identify 'mixing' of columns data in 24 records of df_malware.
# To fix it, following steps are taken: 1. Identify incorrect rows by checking len of IP column,
#   2. Get records into new df
#   3. Rename columns
#   4. Drop incorrect rows from df_malware
#   5. Concatenate fixed data to df_malware

incorrect_rows_idx = df_malware.index[df_malware['IP'].str.len()==2]
df_incorrect_rows = df_malware.iloc[incorrect_rows_idx]

# Applies to df_malware only - rename of columns for 24 records
col_val_replace_to = {
    'Country': 'TTL',
    'TTL': 'Domain',
    'IP': 'Country',
    'Domain': 'IP',
}

df_incorrect_rows.rename(columns=col_val_replace_to, inplace=True) # Apply rename
df_malware.drop(incorrect_rows_idx, axis=0, inplace=True) # Drop from malware df incorrect rows
df_malware = pd.concat([df_malware, df_incorrect_rows], ignore_index=False) # Concatenate fixed data

In [30]:
# To mitigate missing values across similar columns like Domain, Domain_Name and Country, Country_1, following code is applied to df's
# The code also applies mapping to unify a bit entries
countries_map = {
    '-':'',
    "china":"CN",
    "Malaysia":'ID',
    "United States":"US",
    "TURKEY":'TR',
    'RUSSIA':'RU',
    'Russian Federation':'RU',
    'Belarus':'BY',
    'Korea':'KR',
}

def use_regex(input_text):
    return re.sub(r"b'(.+?).'", r"\1", input_text)

def impute_similar_cols(df):
    df["Country_1"].replace(countries_map, inplace=True)
    df["Country"].replace(countries_map, inplace=True)
    df["Country_1"].fillna(df["Country"], inplace=True)
    df["Country"].fillna(df["Country_1"], inplace=True)
    df["Domain_Name"].fillna(df["Domain"].apply(use_regex), inplace=True)
    df['Domain_Age'] = df['Domain_Age'].str.split(' ').str[0]
    return df

df_malware = impute_similar_cols(df_malware)
df_benign = impute_similar_cols(df_benign)

In [31]:
print(df_malware.shape, df_benign.shape)

(4999, 38) (494135, 38)


In [32]:
df_benign['is_threat'] = 0
df_malware['is_threat'] = 1

# Combine two dataframes
df = pd.concat([df_benign, df_malware])
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

# Manually select informative fields based on # of nulls it contain
percent_na = (df.isna().sum() / len(df)) * 100
columns_to_keep = percent_na[percent_na <= 30].index.tolist()
df = df[columns_to_keep]
df.dropna(axis=0, inplace=True)
df.drop_duplicates(inplace=True)

print('df shape: ', df.shape)
print(df.columns)

df shape:  (244072, 35)
Index(['1gram', '2gram', '3gram', 'ASN', 'Alexa_Rank', 'Country', 'Country_1',
       'Creation_Date_Time', 'Domain', 'Domain_Age', 'Domain_Name', 'IP',
       'Name_Server_Count', 'Page_Rank', 'Registrar', 'TTL',
       'char_distribution', 'dec_32', 'dec_8', 'entropy', 'hex_32', 'hex_8',
       'len', 'longest_word', 'numeric_percentage', 'obfuscate_at_sign',
       'oc_32', 'oc_8', 'puny_coded', 'shortened', 'sld', 'subdomain', 'tld',
       'typos', 'is_threat'],
      dtype='object')


In [33]:
# df.to_csv('./tmp/test.csv')

In [34]:
# Manual features selection based on previous experiments, numeric ones only
numerical_features = ['Alexa_Rank','ASN','Domain_Age','TTL','entropy','len','numeric_percentage','subdomain']
categorical_features = ['is_threat']
selected_cols = numerical_features + categorical_features

df = df[selected_cols]

def cols_to_num_drop(df, cols):
    for col in cols:
        df.loc[:,col] = pd.to_numeric(df[col], errors='coerce') # Should be number
        df[col] = df[col].astype('float64')
        df.dropna(subset=[col], inplace=True)

    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def cols_to_num_fill(df, cols):
    for col in cols:
        df.loc[:,col] = pd.to_numeric(df[col], errors='coerce') # Should be number
        df.loc[:,col].fillna(0, inplace = True)

    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def cols_to_cat(df, cols):
    for col in cols:
        df[col] = df[col].astype('category')
    return df

df = cols_to_num_drop(df, numerical_features)
df = cols_to_cat(df, categorical_features)
df.shape

(215120, 9)

In [35]:
print(f"{df['is_threat'].value_counts()[1]/df['is_threat'].value_counts()[0]*100:.3f}% of whole dataset domains are a threat")

1.285% of whole dataset domains are a threat


In [36]:
def train_valid_singleframe_proportional(df, ratio = 0.1):
    train = df.iloc[:int(len(df) * (1 - ratio))]
    valid = df.iloc[int(len(df) * (1 - ratio)):]
    return (train, valid)

def train_valid_singleframe_n_last(df, n_samples = 200):
    train = df.iloc[:int(len(df) - n_samples)]
    valid = df.iloc[int(len(df) - n_samples):]
    return (train, valid)

def get_validset(df, type):
    unique_groups = df['is_threat'].unique()
    ret_train = pd.DataFrame()
    ret_valid = pd.DataFrame()

    for group in unique_groups:
        selected_df = df[df['is_threat']==group]

        if type=='n_last':
            train, valid = train_valid_singleframe_n_last(selected_df)
        elif type=='proportional':
            train, valid = train_valid_singleframe_proportional(selected_df)
        else:
            raise('Only "n_last" or "proportional" is available')

        ret_train = pd.concat([ret_train, train], ignore_index=False)
        ret_valid = pd.concat([ret_valid, valid], ignore_index=False)

    ret_train.reset_index(drop=True, inplace=True)
    ret_valid.reset_index(drop=True, inplace=True)
    return (ret_train, ret_valid)


df, df_valid_n_last = get_validset(df, 'n_last')

In [37]:
X = df.drop(columns='is_threat')
y = df['is_threat']

In [38]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, balanced_accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import optuna

label_encoder = LabelEncoder()
scaler = MinMaxScaler()
sss = StratifiedShuffleSplit(n_splits=25, test_size=0.2, random_state=42)

def objective(trial, cv=sss, cv_fold_func=np.average):
    hparams = {
        # "eval_metric":"error",
        "scale_pos_weight":99,
        'verbosity': 0,

        'n_estimators': trial.suggest_int('n_estimators', 300, 800),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.50, 0.60),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.80, 0.90),
        'gamma': trial.suggest_int('gamma', 13, 20),
    }

    accuracy_scores = []
    balanced_acc_scores = []
    f1_scores = []
    roc_aucs = []

    clf = XGBClassifier(**hparams)

    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]


        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        balanced_acc = balanced_accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        y_prob = clf.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_prob)

        accuracy_scores.append(accuracy)
        balanced_acc_scores.append(balanced_acc)
        f1_scores.append(f1)
        roc_aucs.append(roc_auc)

    print(f'Trial done: balanced accuracy scores on folds: {balanced_acc_scores}')
    print(f'    Trial mean balanced accuracy score: {cv_fold_func(balanced_acc_scores)}')
    print(f'    Trial mean f1 score: {cv_fold_func(f1_scores)}')
    print(f'    Trial mean roc auc score: {cv_fold_func(roc_aucs)}')
    return cv_fold_func(balanced_acc_scores)

In [39]:
n_trials = 20
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=n_trials)
trial = study.best_trial

print(f"Number of finished trials: {len(study.trials)}")
print("Best trial:")
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
    
best_params = trial.params

[I 2023-10-11 14:18:31,784] A new study created in memory with name: no-name-0e5f042b-78c7-4d98-a67f-ab9e4d6e648a
C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.50, 0.60),
C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float 

Trial done: balanced accuracy scores on folds: [0.6897699354762037, 0.6948740566801288, 0.6960978368443377, 0.7115428748230155, 0.695838635185447, 0.7024861844491299, 0.6933337021578282, 0.6958739808662048, 0.6996380397387902, 0.7030383966789403, 0.6888745115636724, 0.6888862934572584, 0.7014847234943252, 0.7028970139559091, 0.6814877765415327, 0.7006497458184523, 0.6666420783670091, 0.7040618846521882, 0.7025553390419169, 0.693357265945, 0.6956854705688298, 0.7032269069763153, 0.6944160995990059, 0.6890630218610474, 0.6910393064460276]
    Trial mean balanced accuracy score: 0.6954728432475406
    Trial mean f1 score: 0.9796393640872956
    Trial mean roc auc score: 0.9102796406829811


C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.50, 0.60),
C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.80, 0.90),
[I 2023-10-11 14:35:22,431] 

Trial done: balanced accuracy scores on folds: [0.8314256910848972, 0.8493474879978362, 0.8315081643399989, 0.820852209975883, 0.839000936404412, 0.8454051639527331, 0.8293198056704717, 0.8331309921174008, 0.8303637838934343, 0.8441183762706517, 0.8336242948792817, 0.8256366832842592, 0.8180189206965865, 0.8221082622833924, 0.8189332980901038, 0.8372116253480781, 0.8238622276589684, 0.8296435516159635, 0.8291620307476687, 0.8220006884723904, 0.8351882131887541, 0.8409710739144778, 0.8299544911553838, 0.8339085970940727, 0.8205192434180195]
    Trial mean balanced accuracy score: 0.8310086325422048
    Trial mean f1 score: 0.895952103722541
    Trial mean roc auc score: 0.9130349291698636


C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.50, 0.60),
C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.80, 0.90),
[I 2023-10-11 14:56:45,699] 

...

Trial done: balanced accuracy scores on folds: [0.8182069187377187, 0.8258938359181783, 0.8175266424471915, 0.8154238305702232, 0.8235128689013334, 0.8306357919583967, 0.8167726012576915, 0.8196074273057166, 0.8073020078395695, 0.8243565549333349, 0.819398426758627, 0.8243847290266927, 0.8148198804598832, 0.8264419500980458, 0.8291031212797391, 0.8172833207318297, 0.82302981126431, 0.8197723738159197, 0.8224084444417123, 0.8232167847929563, 0.821806031100101, 0.8277553751047564, 0.8195029270321718, 0.8273696461538776, 0.8144546417587191]
    Trial mean balanced accuracy score: 0.8211994377475479
    Trial mean f1 score: 0.8337749897021086
    Trial mean roc auc score: 0.905051061702289


C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.50, 0.60),
C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.80, 0.90),
[I 2023-10-11 16:57:14,657] 

Trial done: balanced accuracy scores on folds: [0.7473096302124633, 0.7601539432461063, 0.7595797039978527, 0.7639313207810063, 0.7661417064689767, 0.7548377479576344, 0.7590039279808702, 0.7498970364951838, 0.763190598253821, 0.758120285961925, 0.7559554910795698, 0.7507084503838848, 0.7506024133416114, 0.7583087962593, 0.7384045676864663, 0.7607563565877178, 0.7280518690181277, 0.7720234326495737, 0.7648236711560804, 0.7665438276196272, 0.7551189766349682, 0.7620359726823991, 0.7490871593752113, 0.738852279642732, 0.7467543444451958]
    Trial mean balanced accuracy score: 0.7552077403967323
    Trial mean f1 score: 0.9672187019037619
    Trial mean roc auc score: 0.9155303901138643


C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.50, 0.60),
C:\Users\rafal\AppData\Local\Temp\ipykernel_8480\1596549850.py:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.80, 0.90),
[I 2023-10-11 17:12:20,473] 

Trial done: balanced accuracy scores on folds: [0.6752218581787857, 0.6801845966596795, 0.6933721213760432, 0.6931247016107385, 0.6931718291850824, 0.6826203750945112, 0.674585635925145, 0.6813965949303025, 0.6782672215426289, 0.697466073269035, 0.665680573399148, 0.6825496837329956, 0.6930068826748792, 0.6834317889832123, 0.6806912180838748, 0.6750907205806117, 0.6637867620692693, 0.6915607833012661, 0.6950302948342031, 0.6875139589826181, 0.6683033253626263, 0.6763734102127503, 0.6816911422699509, 0.6675743847290267, 0.6757269428342523]
    Trial mean balanced accuracy score: 0.6814969151929055
    Trial mean f1 score: 0.9819383767942105
    Trial mean roc auc score: 0.9164650146682246
Number of finished trials: 20
Best trial:
  Value: 0.8353884234187163
  Params: 
    n_estimators: 521
    max_depth: 4
    learning_rate: 0.06227012530557216
    subsample: 0.5823180676644505
    colsample_bytree: 0.8679467785467245
    gamma: 14


## References

Samaneh Mahdavifar, Nasim Maleki, Arash Habibi Lashkari, Matt Broda, Amir H. Razavi, “Classifying Malicious Domains using DNS Traffic Analysis”, The 19th IEEE International Conference on Dependable, Autonomic, and Secure Computing (DASC), Oct. 25-28, 2021, Calgary, Canada